In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import LabelEncoder
from keras.layers import LSTM,Dense,Dropout, Reshape
from keras.models import Sequential
from keras.layers import Embedding,SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

from tensorflow.keras.optimizers import Adam

from gensim.models import Word2Vec

In [40]:
cleaned_lyrics = pd.read_csv('Data/tcc_ceds_music.csv')

In [2]:
from google.cloud import storage
import os
import io
client = storage.Client()
bucket = client.get_bucket('nlp_final_data')

blob = bucket.blob('tcc_ceds_music.csv')
content = blob.download_as_string()

df = pd.read_csv(io.BytesIO(content))

In [4]:
# # separate the dataset into the target variable and features
X = cleaned_lyrics['lyrics']
y = cleaned_lyrics['genre']


In [5]:
# split the balanced data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2,stratify=y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

In [6]:
tokenize = Tokenizer(char_level=False)
tokenize.fit_on_texts(X_train) # only fit on train

# we will need this later
num_words = len(tokenize.word_index)+1

MemoryError: Unable to allocate 1.08 TiB for an array with shape (346632, 428962) and data type float64

In [7]:
num_words

428962

In [ ]:
x_train = tokenize.texts_to_matrix(X_train)
x_test = tokenize.texts_to_matrix(X_test)

encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

num_classes = np.max(y_train) + 1
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [23]:
batch_size = 32
epochs = 2

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(num_words,),activation='relu'))
model.add(keras.layers.Reshape((64, 1)))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/2
639/639 [==============================] - 89s 138ms/step - loss: 1.5887 - accuracy: 0.3766 - val_loss: 1.5164 - val_accuracy: 0.4286
Epoch 2/2
639/639 [==============================] - 83s 129ms/step - loss: 1.0928 - accuracy: 0.6004 - val_loss: 1.6001 - val_accuracy: 0.4110


In [51]:
model3 = Sequential([
    Dense(64, input_shape=(num_words,),activation='relu'),
    Reshape((64, 1)),
    LSTM(units=64,recurrent_dropout=0.5),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# compile the model
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [53]:
model3.fit(x_train, y_train,
          batch_size=32,
          epochs=3,
          verbose=1,
          validation_split=0.2)

Epoch 1/3
568/568 [==============================] - 40s 71ms/step - loss: 1.3459 - accuracy: 0.4750 - val_loss: 1.7296 - val_accuracy: 0.3575
Epoch 2/3
568/568 [==============================] - 38s 67ms/step - loss: 1.0873 - accuracy: 0.6022 - val_loss: 1.9313 - val_accuracy: 0.3513
Epoch 3/3
568/568 [==============================] - 40s 70ms/step - loss: 0.8469 - accuracy: 0.7076 - val_loss: 1.9702 - val_accuracy: 0.3524


In [ ]:
all_songs = pd.read_csv('song_lyrics.csv')

In [ ]:
songs_sample = all_songs.sample()